### Predictive Model with SVM

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

#Load libraries for data processing
import pandas as pd #data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
from scipy.stats import norm

## Supervised learning.
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix
from sklearn import metrics, preprocessing
from sklearn.metrics import classification_report

from scipy.io import loadmat 

# visualization
import seaborn as sns 
plt.style.use('fivethirtyeight')
sns.set_style("white")

plt.rcParams['figure.figsize'] = (8,4)

In [2]:
xx=loadmat('./wk_30_new.mat')

In [3]:
xx

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Tue Oct 31 01:59:05 2023',
 '__version__': '1.0',
 '__globals__': [],
 'wk_30': array([[ 3.09123000e-01,  1.89735000e-01,  2.65190000e-02, ...,
          2.83401190e+01,  7.53646780e+01,  2.00000000e+00],
        [-2.18090000e-02,  6.43510000e-02,  3.30059000e-01, ...,
          2.82928060e+01,  7.51611860e+01,  1.00000000e+00],
        [ 2.12515000e-01, -1.08309000e-01,  4.31239000e-01, ...,
          2.82928060e+01,  7.54016760e+01,  2.00000000e+00],
        ...,
        [-1.56420000e-02,  6.64060000e-02,  1.28757823e+02, ...,
          0.00000000e+00,  7.54325080e+01,  2.00000000e+00],
        [-8.75840000e-02, -5.89780000e-02,  1.28535226e+02, ...,
          0.00000000e+00,  7.55805020e+01,  2.00000000e+00],
        [-9.99170000e-02,  1.42459000e-01,  1.28383456e+02, ...,
          0.00000000e+00,  7.54386750e+01,  2.00000000e+00]])}

In [ ]:
xx.keys()

In [4]:
df=xx['wk_30']
df2=xx['wk_30']

In [5]:
X=df[:,:8]

In [6]:
y=df2[:,-1]

In [7]:
X.shape

(6000, 8)

In [8]:
y.shape

(6000,)

In [9]:
X

array([[ 3.09123000e-01,  1.89735000e-01,  2.65190000e-02, ...,
         2.96704654e+03,  2.85293680e+01,  2.83401190e+01],
       [-2.18090000e-02,  6.43510000e-02,  3.30059000e-01, ...,
         2.96705440e+03,  2.84820560e+01,  2.82928060e+01],
       [ 2.12515000e-01, -1.08309000e-01,  4.31239000e-01, ...,
         2.96706226e+03,  2.84347440e+01,  2.82928060e+01],
       ...,
       [-1.56420000e-02,  6.64060000e-02,  1.28757823e+02, ...,
         2.99578712e+03,  0.00000000e+00,  0.00000000e+00],
       [-8.75840000e-02, -5.89780000e-02,  1.28535226e+02, ...,
         2.99578712e+03,  0.00000000e+00,  0.00000000e+00],
       [-9.99170000e-02,  1.42459000e-01,  1.28383456e+02, ...,
         2.99578712e+03,  0.00000000e+00,  0.00000000e+00]])

In [10]:
y

array([2., 1., 2., ..., 2., 2., 2.])

In [12]:
import collections
frequency = collections.Counter(y)
print(dict(frequency))

{2.0: 3962, 1.0: 2038}


In [13]:
from scipy import stats
stats.describe(X)

DescribeResult(nobs=6000, minmax=(array([-5.03305400e+00, -3.49367400e+00, -5.19853000e-01, -3.84444000e-01,
        3.07729920e+03,  2.96704654e+03, -2.74411800e+00, -2.50755600e+00]), array([   6.837303,    4.134187,  141.071436,  142.993542, 3106.082675,
       2995.787596,   28.623993,   28.623993])), mean=array([-1.66863492e-02, -5.39768400e-03,  3.25982676e+01,  3.39340208e+01,
        3.09705385e+03,  2.98678963e+03,  1.72745720e+01,  1.72490706e+01]), variance=array([2.12902525e-01, 1.30884768e-01, 2.26892049e+03, 2.37377516e+03,
       8.74650140e+01, 8.73161308e+01, 1.58821322e+02, 1.58858465e+02]), skewness=array([ 0.85503852,  0.00358867,  1.27360007,  1.21416113, -0.64888808,
       -0.64987555, -0.4198439 , -0.41743412]), kurtosis=array([32.56987838, 16.37916128,  0.06479804, -0.11098028, -1.03009386,
       -1.02947318, -1.66615633, -1.66962919]))

In [14]:
#transform the class labels from their original string representation (M and B) into integers
le = LabelEncoder()
y = le.fit_transform(y)

# Normalize the  data (center around 0 and scale to remove the variance).
scaler =StandardScaler()
Xs = scaler.fit_transform(X)

In [15]:
Xs

array([[ 0.70617011,  0.53941331, -0.68386067, ..., -2.11302157,
         0.89313975,  0.88004244],
       [-0.01110301,  0.19280916, -0.6774877 , ..., -2.11218046,
         0.88938525,  0.87628829],
       [ 0.49677869, -0.28448199, -0.67536337, ..., -2.11133934,
         0.88563074,  0.87628829],
       ...,
       [ 0.00226356,  0.19848988,  2.0189195 , ...,  0.96296522,
        -1.37084731, -1.36866359],
       [-0.15366595, -0.14811427,  2.01424596, ...,  0.96296522,
        -1.37084731, -1.36866359],
       [-0.1803969 ,  0.40872632,  2.01105947, ...,  0.96296522,
        -1.37084731, -1.36866359]])

In [16]:
# compare the number of repeats for repeated k-fold cross-validation
from scipy.stats import sem
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot

In [ ]:
# evaluate a model with a given number of repeats
clf = SVC(probability=True)
def evaluate_model_acc(X, y, repeats):
    # prepare the cross-validation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=repeats, random_state=1)
    # create model
    # model = LogisticRegression()
    # evaluate model
    scores = cross_val_score(clf, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    return scores
 
def evaluate_model_apr(X, y, repeats):
    # prepare the cross-validation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=repeats, random_state=1)
    # create model
    # model = LogisticRegression()
    # evaluate model
    scores = cross_val_score(clf, X, y, scoring='average_precision', cv=cv, n_jobs=-1)
    return scores

def evaluate_model_f1(X, y, repeats):
    # prepare the cross-validation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=repeats, random_state=1)
    # create model
    # model = LogisticRegression()
    # evaluate model
    scores = cross_val_score(clf, X, y, scoring='f1', cv=cv, n_jobs=-1)
    return scores

def evaluate_model_recall(X, y, repeats):
    # prepare the cross-validation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=repeats, random_state=1)
    # create model
    # model = LogisticRegression()
    # evaluate model
    scores = cross_val_score(clf, X, y, scoring='recall', cv=cv, n_jobs=-1)
    return scores

# configurations to test
repeats = range(1,4)
results_acc = list()
results_apr = list()
results_f1 = list()
results_recall = list()

for r in repeats:
    # evaluate using a given number of repeats
    scores_acc = evaluate_model_acc(Xs, y, r)
    scores_apr = evaluate_model_apr(Xs, y, r)
    scores_f1 = evaluate_model_f1(Xs, y, r)
    scores_recall = evaluate_model_recall(Xs, y, r)
    # summarize
    print('>%d mean=%.4f se=%.3f std=%.3f' % (r, mean(scores_acc), sem(scores_acc), std(scores_acc)))
    print('>%d mean=%.4f se=%.3f std=%.3f' % (r, mean(scores_apr), sem(scores_apr), std(scores_apr)))
    print('>%d mean=%.4f se=%.3f std=%.3f' % (r, mean(scores_f1), sem(scores_f1), std(scores_f1)))
    print('>%d mean=%.4f se=%.3f std=%.3f' % (r, mean(scores_recall), sem(scores_recall),std(scores_recall)))
    # store
    results_acc.append(scores_acc)
    results_apr.append(scores_apr)
    results_f1.append(scores_f1)
    results_recall.append(scores_recall)
# plot the results
pyplot.boxplot(results_acc, labels=[str(r) for r in repeats], showmeans=True)
pyplot.show()

In [ ]:
clf = SVC(probability=True)
X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.3, random_state=2, stratify=y)
y_pred = clf.fit(X_train, y_train).predict(X_test)
cm = metrics.confusion_matrix(y_test, y_pred)
cm

In [ ]:
print(classification_report(y_test, y_pred ))